In [1]:
!pip install keras
!pip install tensorflow==2.2
!pip install opencv-python


In [2]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
import webbrowser
import subprocess
            
# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)
#whitebg = cv2.imread('"C:\\Users\\Hp\\Pictures\\AIphotos\\whitebg.jpg"')
# Category dictionary
#categories = {0: 'ZERO', 1: 'ONE', 2: 'TWO', 3: 'THREE', 4: 'FOUR', 5: 'FIVE'}
categories = {0: 'A', 1: 'B', 2: 'C',3: 'D', 4: 'E', 5: 'F',6: 'G', 7: 'H', 8: 'I',
              9: 'J', 10: 'K', 11: 'L',12: 'M', 13: 'N', 14: 'O',15: 'P', 16: 'Q',
              17: 'R', 18: 'S',19: 'T', 20: 'U', 21: 'V',22: 'W', 23: 'X', 24: 'Y',25: 'Z', 
              26: 'SPACE', 27: 'REMOVE 1', 28: 'REMOVE ALL', 29: 'BLANK'}
s=""
d={}
p=""
count=0

while True:
    count+=1
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    # Got this from collect-data.py
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    
    # Resizing the ROI so it can be fed to the model for prediction
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    # Batch of 1
    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    prediction = {'A': result[0][0], 
                  'B': result[0][1], 
                  'C': result[0][2],
                  'D': result[0][3],
                  'E': result[0][4],
                  'F': result[0][5],
                  'G': result[0][6], 
                  'H': result[0][7], 
                  'I': result[0][8],
                  'J': result[0][9], 
                  'K': result[0][10], 
                  'L': result[0][11],
                  'M': result[0][12],
                  'N': result[0][13],
                  'O': result[0][14],
                  'P': result[0][15], 
                  'Q': result[0][16], 
                  'R': result[0][17], 
                  'S': result[0][18],
                  'T': result[0][19], 
                  'U': result[0][20], 
                  'V': result[0][21],
                  'W': result[0][22],
                  'X': result[0][23],
                  'Y': result[0][24],
                  'Z': result[0][25],
                  'SPACE': result[0][26],
                  'REMOVE 1': result[0][27],
                  'REMOVE ALL': result[0][28],
                  'BLANK': result[0][29]}
                
                
    # Sorting based on top prediction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    
    # Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
    #cv2.Waitkey(30)
    #s+=prediction[0][0]
    if count<=500:
        if prediction[0][0] in d:
            d[prediction[0][0]]+=1
        else:
            d[prediction[0][0]]=1
        count+=1
    
    if count>500:
        inverse = [(value, key) for key, value in d.items()]
        a=max(inverse)[1]
        if a=='SPACE':
            s+=' '
            cv2.putText(frame, 'SPACE PRINTED' , (10, 100), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
            cv2.waitKey(1000)
        elif a=='REMOVE 1':
            s=s[:-1]
        elif a=='REMOVE ALL':
            s=''
        elif a!='BLANK':
            s+=a
        d={}
        count=0
         
    cv2.putText(frame, s, (10, 400), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
    cv2.imshow("Frame", frame)
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
#print(s)
cap.release()
cv2.destroyAllWindows()


Loaded model from disk
